In [ ]:
import pandas as pd

# Load item datasets
df1 = pd.read_csv('Items/Items/CDs_and_Vinyls_items_processed.csv')
df2 = pd.read_csv('Items/Items/Digital_Music_items_processed.csv')
df3 = pd.read_csv('Items/Items/Magazine_Subscriptions_items_processed.csv')
df4 = pd.read_csv('Items/Items/Movies_and_TV_items_processed.csv')
df5 = pd.read_csv('Items/Items/Musical_Instruments_items_processed.csv')
df6 = pd.read_csv('Items/Items/Sports_and_Outdoors_items_processed.csv', low_memory=False)
df7 = pd.read_csv('Items/Items/Toys_and_Games_items_processed.csv')
df8 = pd.read_csv('Items/Items/Video_Games_items_processed.csv')

# Combine item datasets
items_df = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], ignore_index=True)

# Load review datasets in a similar manner
reviews_df1 = pd.read_csv('Reviews/Reviews/CDs_and_Vinyl_reviews_processed.csv')
reviews_df2 = pd.read_csv('Reviews/Reviews/Digital_Music_reviews_processed.csv')
reviews_df3 = pd.read_csv('Reviews/Reviews/Magazine_Subscriptions_reviews_processed.csv')
reviews_df4 = pd.read_csv('Reviews/Reviews/Movies_and_TV__reviews_processed.csv')
reviews_df5 = pd.read_csv('Reviews/Reviews/Musical_Instruments_reviews_processed.csv')
reviews_df6 = pd.read_csv('Reviews/Reviews/Sports_and_Outdoors_reviews_processed.csv', low_memory=False)
reviews_df7 = pd.read_csv('Reviews/Reviews/Toys_and_Games_reviews_processed.csv')
reviews_df8 = pd.read_csv('Reviews/Reviews/Video_Games_reviews_processed.csv')

# Combine review datasets
reviews_df = pd.concat([reviews_df1, reviews_df2, reviews_df3, reviews_df4, reviews_df5, reviews_df6, reviews_df7, reviews_df8], ignore_index=True)

# display(items_df.head())
# display(reviews_df.head())

In [64]:

# Merge datasets on 'parent_asin'
merged_df = pd.merge(reviews_df, items_df, on='parent_asin', how='inner')

# Remove duplicate reviews
merged_df = merged_df.drop_duplicates(subset=['user_id', 'review_text', 'date'])

# Standardize column names
merged_df.columns = [col.lower().replace(' ', '_') for col in merged_df.columns]

# Convert 'date' to datetime
merged_df['date'] = pd.to_datetime(merged_df['date'])

# Filter to include only records from 2023
merged_df_2023 = merged_df[merged_df['date'].dt.year == 2023]

# Convert 'price' to numeric using .loc to avoid SettingWithCopyWarning
merged_df_2023.loc[:, 'price'] = pd.to_numeric(merged_df_2023['price'], errors='coerce')

# Handle missing data
merged_df_2023 = merged_df_2023.dropna(subset=['item_average_rating', 'item_rating_number', 'price'])

# Validate data
merged_df_2023 = merged_df_2023[(merged_df_2023['price'] > 0) & (merged_df_2023['item_average_rating'].between(1, 5))]

# Drop unnecessary columns
merged_df_2023 = merged_df_2023.drop(columns=['helpful_vote', 'store', 'features', 'review_text','details','description','parent_asin','main_category_y'])


In [65]:
# Export the cleaned and filtered data for 2023
merged_df_2023.to_csv('cleaned_merged_data_2023.csv', index=False)

